## Web Scraping Challenge

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pymongo
import requests
import GetOldTweets3 as got
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url =  "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(2)

html = browser.html
soup = bs(html, "lxml")

cont = soup.find('div', class_="image_and_description_container")
news_title = cont.find('div', class_="content_title").text
news_p = cont.find("div", class_="article_teaser_body").text

In [4]:
news_title

"How NASA's Mars Helicopter Will Reach the Red Planet's Surface"

In [5]:
news_p

'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.'

## JPL Mars Space Images - Featured Image

In [6]:
url =  "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
browser.click_link_by_partial_text("FULL IMAGE")
browser.click_link_by_partial_text("more info")

html = browser.html
soup = bs(html, "html.parser")

featured_image = browser.find_link_by_partial_text(".jpg")["href"]

print(featured_image)

/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


https://photojournal.jpl.nasa.gov/jpeg/PIA18433.jpg


## Mars Weather

In [7]:
url =  "https://twitter.com/marswxreport?lang=en"

In [8]:
tweetCriteria = got.manager.TweetCriteria().setUsername("MarsWxReport").setMaxTweets(1)
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

user_tweet = [tweet.text for tweet in tweets]
mars_weather = user_tweet[0]
mars_weather

'InSight sol 561 (2020-06-25) low -89.7ºC (-129.5ºF) high -2.9ºC (26.8ºF) winds from the W at 5.7 m/s (12.8 mph) gusting to 17.8 m/s (39.8 mph) pressure at 7.60 hPa'

## Mars Facts

In [9]:
url =  "https://space-facts.com/mars/"
tables = pd.read_html(url)

In [10]:
facts_table = tables[0]
facts_table.rename(columns={0:"Description", 1:"Value"}, inplace=True)
facts_table

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [11]:
Facts = facts_table.to_html(index=False)

## Mars Hemispheres

In [12]:
url =  "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

html = browser.html
soup = bs(html, "html.parser")

In [13]:
items = soup.find_all('div', class_='description')
hemisphere_image_urls = []
for x in items:
    images_dict = {}
    title = x.a.h3.text
    browser.click_link_by_partial_text(title)
    image_url = browser.find_link_by_partial_text('Sample')['href']
    images_dict['title'] = title
    images_dict['image_url'] = image_url
    hemisphere_image_urls.append(images_dict)
    browser.back()
print(hemisphere_image_urls)

/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


[{'title': 'Cerberus Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [14]:
browser.quit()

info_mars = {"news_title": news_title,
    "news_p":news_p,
    "featured_image":featured_image,
    "mars_weather":mars_weather,
    "Facts":Facts,
    "hemisphere_image_urls":hemisphere_image_urls}

In [15]:
info_mars

{'news_title': "How NASA's Mars Helicopter Will Reach the Red Planet's Surface",
 'news_p': 'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.',
 'featured_image': 'https://photojournal.jpl.nasa.gov/jpeg/PIA18433.jpg',
 'mars_weather': 'InSight sol 561 (2020-06-25) low -89.7ºC (-129.5ºF) high -2.9ºC (26.8ºF) winds from the W at 5.7 m/s (12.8 mph) gusting to 17.8 m/s (39.8 mph) pressure at 7.60 hPa',
 'Facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n    